In [28]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import datetime
from fake_useragent import UserAgent, FakeUserAgentError

# Словарь для замены названий месяцев на числовые значения
dict_months = {
    "января": "1",
    "февраля": "2",
    "марта": "3",
    "апреля": "4",
    "мая": "5",
    "июня": "6",
    "июля": "7",
    "августа": "8",
    "сентября": "9",
    "октября": "10",
    "ноября": "11",
    "декабря": "12"
}

def parse(p: int = 1):
    """Парсит данные с указанной страницы каталога товаров."""
    p = str(p)  # Преобразуем номер страницы в строку
    res = []  # Список для хранения результатов парсинга

    # Настройки для использования Firefox
    firefox_options = Options()

    firefox_options.add_argument("--headless")  # Можно раскомментировать для фона (без интерфейса)

    # Обработка пользовательского агента
    try:
        ua = UserAgent().random  # Генерация случайного пользовательского агента
    except FakeUserAgentError:
        # Если не удалось сгенерировать пользовательский агент, используем стандартный
        ua = "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:84.0) Gecko/20100101 Firefox/84.0"

    # Установка пользовательского агента в настройки Firefox
    firefox_options.set_preference("general.useragent.override", ua)

    # Создаем экземпляр сервиса FirefoxDriver
    service = Service()  # Убедитесь, что geckodriver доступен в PATH
    # Инициализируем драйвер Firefox
    driver = webdriver.Firefox(service=service, options=firefox_options)

    try:
        # Переход на страницу с товарами
        driver.get(f'https://www.wildberries.ru/catalog/yuvelirnye-ukrasheniya/chasy?page={p}')
        try:
            # Ожидание, пока элементы товаров не появятся на странице
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".product-card"))
            )
        except:
            return []  # Если товары не появились, возвращаем пустой список

        # Получение всех элементов с товарами
        cars = driver.find_elements(By.CSS_SELECTOR, '.product-card')  # Проверьте правильность селектора
        # Проходимся по всем предложениям
        for car in cars:
            buf = []  # Временный список для хранения данных о товаре
            buf.append(car.find_element(By.CSS_SELECTOR, ".product-card__link").get_attribute("href").split("/")[-2])  # ID товара
            buf.append(car.find_element(By.CSS_SELECTOR, '.product-card__name').text)  # Название товара
            buf.append(car.find_element(By.CSS_SELECTOR, '.price').text)  # Цена товара
            data = car.find_element(By.CSS_SELECTOR, '.btn-text').text  # Дата

            # Обработка даты
            date_processed = process_date(data)  # Преобразуем дату в нужный формат
            buf.append(date_processed)  # Добавляем обработанную дату в список
            res.append(buf)  # Добавляем данные о товаре в общий список результатов

    finally:
        driver.quit()  # Закрываем браузер, освобождая ресурсы

    return res  # Возвращаем результаты парсинга

def process_date(data: str) -> str:
    """Обрабатывает строку даты и возвращает дату в формате MM.DD.YY."""
    # Обработка даты
    if data == "Послезавтра":
        data = datetime.date.today() + datetime.timedelta(days=2)  # Дата послезавтра
    elif data == "Завтра":
        data = datetime.date.today() + datetime.timedelta(days=1)  # Дата завтра
    else:
        # Разделяем строку даты на составляющие
        arr = data.split()
        d = int(arr[0])  # День
        m = int(dict_months[arr[1]])  # Месяц
        a = datetime.datetime.today().year  # Текущий год
        # Если дата уже прошла, увеличиваем год на 1
        data = datetime.date(a, m, d) if datetime.datetime(a, m, d) > datetime.datetime.today() else datetime.date(a + 1, m, d)

    return data.strftime("%m.%d.%y")  # Форматируем дату в MM.DD.YY

def main():
    """Основная функция, запускающая парсинг и записывающая данные в файл."""
    result = []  # Список для хранения всех товаров
    n = 1  # Начинаем с первой страницы
    while True:
        buf = parse(n)  # Парсим данные с текущей страницы
        if not buf:  # Если данных нет, выходим из цикла
            break
        result += buf  # Добавляем все товары на странице в общий список
        n += 1  # Переход к следующей странице

    # Открываем файл с указанием кодировки utf-8 и добавляем заголовки
    with open("data.csv", "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)  # Создаем объект для записи в CSV
        # Записываем заголовки столбцов
        writer.writerow(["ID", "Название", "Цена", "Дата"])
        writer.writerows(result)  # Записываем данные о товарах

if __name__ == "__main__":
    main()  # Запускаем основную функцию


In [29]:
def filter_by_date(row, days_range=7):
    """Проверяет, попадает ли дата из строки в диапазон ближайших days_range дней."""
    date_parts = row[-1].split(".")
    parsed_date = datetime.datetime(int("20" + date_parts[-1]), int(date_parts[0]), int(date_parts[1]))
    return parsed_date <= datetime.datetime.today() + datetime.timedelta(days=days_range)

def process_csv():
    with open('data.csv', 'r', newline='') as csvfile:
        csv_reader = csv.reader(csvfile, delimiter=',')
        next(csv_reader)  # Пропускаем заголовок
        for row in csv_reader:
            if filter_by_date(row):
                print(", ".join(row))

if __name__ == "__main__":
    process_csv()

17995342, / Часы наручные из золота 585 и стали с бриллиантами, 27 437 ₽ 159 990 ₽
с WB кошельком, 10.06.24
4936195, / Часы наручные из золота 585 пробы, 238.01.00.000.03.01.2, 63 305 ₽ 244 990 ₽, 10.07.24
39713385, / Часы наручные из золота 585 и стали с бриллиантами, 27 700 ₽ 139 990 ₽
с WB кошельком, 10.06.24
17995344, / Часы наручные из золота 585 и стали с бриллиантами, 27 700 ₽ 139 990 ₽
с WB кошельком, 10.06.24
17995337, / Часы наручные из золота 585 и стали, 35 385 ₽ 184 990 ₽
с WB кошельком, 10.06.24
82311040, / Часы наручные мужские из серебра 925, 23 544 ₽ 104 990 ₽
с WB кошельком, 10.06.24
33057475, / Часы наручные женские из серебра 925, 7 846 ₽ 34 990 ₽
с WB кошельком, 10.06.24
82298930, / Часы наручные мужские из серебра 925, 21 104 ₽ 99 990 ₽
с WB кошельком, 10.06.24
15113622, / Часы наручные из золота 585 и стали 157.01.71.000.05.01.3, 20 577 ₽ 119 990 ₽
с WB кошельком, 10.07.24
39713375, / Часы наручные из золота 585 и стали с бриллиантами, 27 701 ₽ 149 990 ₽
с WB кош